This code, requested by Paul, was intended to accomplish two (three) tasks. We need to produce all-encompassing, humongous database files containing the following.

* A `positions` database containing the position of all objects, players, etc. across all participants. The data should be modded to include:
    * The position of the user in world space
* A `eeg` database containing all eeg data across all participants. The data should be modded such that:
    * Any rows with gaps in the data should be filtered out.
* A `eye` database containing all eye data across all participants. The data should be modded to include:
    * Local data timestamp.
    * The distance between the user and the gaze target.

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import datetime

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# constants
_INPUT_DATA_DIR = '../data/'
_OUTPUT_DATA_DIR = f'./geosim_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}/'
_PARTICIPANTS = ['P0','P1','P2','P3','P4','P5','P6']
_ROAD_RANGE = [2.25, 7.75]

# Helper function to create a directory if not exists
def try_make_dir(a_dir:str):
    try: os.makedirs(a_dir)
    except OSError as e:
        if e.errno != errno.EEXIST: raise  # This was not a "directory exist" error..

# Helper function for detecting files, from '../helpers.py'
def get_immediate_files(a_dir:str):
    return [name for name in os.listdir(a_dir)
            if os.path.isfile(os.path.join(a_dir, name))]

# Helper function to parse raycast hit names. Does so by process raycast_hit_name 
#   into different segments, based on delimiters.
def raycast_processing(raw_value):
    if len(raw_value) == 0: return '','','',''
    divided = raw_value.split("-")
    values = divided[0].split('.')
    values.append(divided[1])
    return values

In [3]:
try_make_dir(_OUTPUT_DATA_DIR)

# Positions & On-Roads

The idea is that we want to label all our data by whether the user is on the road or not. This would indicate that someone is perhaps crossing or not. We also need to process the positions to include the necessary data.

In [7]:
# Initialize multiple lists to store all the positions, player positions, and on_road data.
# We will concatenate these later into huge dataframes
trials_dfs = []
positions_dfs = []
players_dfs = []
on_roads_dfs = []

# Iterate through participants
for p in _PARTICIPANTS:
    # Get the neceesary files, check if file exists or not
    pdir = os.path.join(_INPUT_DATA_DIR, p)
    pfiles = get_immediate_files(pdir)
    if "positions_mod.csv" not in pfiles:
        print(f"ERROR: 'positions_mod.csv' not available in {p}'s data'")
        continue
        
    # Read the trial data for this participant.
    # We simply need this to restrict the eye gazes between a time range.
    tdf = pd.read_csv(os.path.join(pdir,'trials.csv'))
    tdf = tdf[tdf['trial_number']>=1]
    start_frame = tdf['start_frame'].iloc[0]
    end_frame = tdf['end_frame'].iloc[-1]
    
    # Read the csv file, filter to only the trial data
    pos_df = pd.read_csv(os.path.join(pdir,'positions_mod.csv'))
    pos_df = pos_df[pos_df['frame'].between(start_frame, end_frame)]
    
    # Create a new timestamp col for the local timestamp
    pos_df['local_timestamp'] = pos_df['unix_ms'].apply(lambda t: datetime.datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))
    
    # Add the participant ID to each row
    pos_df['participant'] = p
    
    # Add the trial number based on frame number and trials
    trial_intervals = pd.IntervalIndex.from_arrays(tdf['start_frame'], tdf['end_frame'], closed='left')
    pidx = trial_intervals.get_indexer(pos_df['frame'])
    pos_df['trial_number'] = tdf['trial_number'].iloc[pidx].where(pidx != -1).values
    
    # Create a player-specific dataframe
    player_df = pos_df[pos_df['obj_name'] == 'CenterEyeAnchor']
    player_df = player_df.reset_index(drop=True)
    
    # Mod the player-specific dataframe to include speed and displacement
    player_df['time_diff'] = player_df['unix_ms'].diff() / 1000 # unix ms to sec
    player_df['displacement'] = np.sqrt((player_df['pos_x'].diff()**2) + (player_df['pos_z'].diff()**2))
    player_df['speed'] = player_df['displacement'] / player_df['time_diff']
    # Make sure we don't have NaN values in the first sample row
    player_df.iloc[0, player_df.columns.get_loc('time_diff')] = 0
    player_df.iloc[0, player_df.columns.get_loc('displacement')] = 0
    player_df.iloc[0, player_df.columns.get_loc('speed')] = 0
    
    # Calculate crossing moments
    moving_df = player_df[player_df['pos_z'].between(_ROAD_RANGE[0], _ROAD_RANGE[1])]
    moving_df['next_frame'] = moving_df['frame'].shift(-1)
    moving_df['frame_diff'] = moving_df['next_frame'] - moving_df['frame']
    moving_df['group'] = (moving_df['next_frame'] != moving_df['frame'] + 1).cumsum()
    moving_df = moving_df[moving_df['frame_diff']==1]
    
    # Crossings defined by combinin g disparate movement segments
    crossing_df = moving_df.groupby('group').agg(
        start_unix_ms=('unix_ms', 'first'),
        start_frame=('frame','first'),
        start_rel_timestamp=('rel_timestamp','first'),
        start_local_timestamp=('local_timestamp','first'),
        end_unix_ms=('unix_ms', 'last'),
        end_frame=('frame','last'),
        end_rel_timestamp=('rel_timestamp','last'),
        end_local_timestamp=('local_timestamp','last'),
        group=('group','first')
    )
    crossing_df = crossing_df.reset_index(drop=True)
    crossing_df = crossing_df.drop(columns=['group'])
    crossing_df['participant'] = p
    cidx = trial_intervals.get_indexer(crossing_df['start_frame'])
    crossing_df['trial_number'] = tdf['trial_number'].iloc[cidx].where(cidx != -1).values
    
    # Label each position based on whether the user is crossing or not.
    crossing_intervals = pd.IntervalIndex.from_arrays(crossing_df['start_frame'], crossing_df['end_frame'], closed='both')
    player_df['on_road'] = player_df['frame'].apply(lambda frame: any(frame in interval for interval in crossing_intervals))
    
    # Add everything to our list of dataframes
    trials_dfs.append(tdf)
    positions_dfs.append(pos_df)
    players_dfs.append(player_df)
    on_roads_dfs.append(crossing_df)

In [8]:
# Let's concatenate everything together into their massive dataframes
positions_df = pd.concat(positions_dfs, ignore_index=True)
players_df = pd.concat(players_dfs, ignore_index=True)
on_roads_df = pd.concat(on_roads_dfs, ignore_index=True)

# Let's save our dataframes
positions_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'positions_all.csv'))
players_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'player_positions_all.csv'))
on_roads_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'on_roads_all.csv'))

display(positions_df)
display(players_df)
display(on_roads_df)

,unix_ms,rel_timestamp,frame,obj_name,guid,pos_x,pos_y,pos_z,forward_x,forward_y,forward_z,local_timestamp,participant,trial_number
0,1742842039794,27.37964,1271,HighMale2,29686,55.194620,0.025000,-1.829232,0.994431,0.000000,-1.053876e-01,2025-03-24 14:47:19.794000,P0,1
1,1742842039794,27.37964,1271,CenterEyeAnchor,27386,0.023046,1.510251,7.537932,0.018721,0.083370,9.963428e-01,2025-03-24 14:47:19.794000,P0,1
2,1742842039794,27.37964,1271,LowFemale2,33854,49.259030,0.025000,11.288620,0.999992,0.000000,4.029572e-03,2025-03-24 14:47:19.794000,P0,1
3,1742842039794,27.37964,1271,HighFemale2,33816,-48.015770,0.025000,-1.138340,-0.998273,0.000000,-5.873787e-02,2025-03-24 14:47:19.794000,P0,1
4,1742842039794,27.37964,1271,LowMale2,28708,-57.869670,0.025000,13.540730,-1.000000,0.000000,5.820990e-04,2025-03-24 14:47:19.794000,P0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464397,1743103505858,257.39490,13182,Head-Remy-Car1-Detailed,-288314,56.269520,1.079971,4.243897,0.928800,-0.370325,-1.381822e-02,2025-03-27 15:25:05.858000,P6,8
1464398,1743103505858,257.39490,13182,Remy-Car2-Detailed,-282146,23.699140,0.271200,5.766800,-1.000000,0.000000,-1.192093e-07,2025-03-27 15:25:05.858000,P6,8
1464399,1743103505858,257.39490,13182,Remy-SportCar5-Detailed,-284356,-38.634860,0.220800,4.209200,1.000000,0.000000,-1.192093e-07,2025-03-27 15:25:05.858000,P6,8
1464400,1743103505858,257.39490,13182,Remy-Car1-Detailed,-288120,56.391880,0.176000,4.233200,1.000000,0.000000,-1.192093e-07,2025-03-27 15:25:05.858000,P6,8


,unix_ms,rel_timestamp,frame,obj_name,guid,pos_x,pos_y,pos_z,forward_x,forward_y,forward_z,local_timestamp,participant,trial_number,time_diff,displacement,speed,on_road
0,1742842039794,27.37964,1271,CenterEyeAnchor,27386,0.023046,1.510251,7.537932,0.018721,0.083370,0.996343,2025-03-24 14:47:19.794000,P0,1,0.000,0.000000,0.000000,True
1,1742842039819,27.40971,1272,CenterEyeAnchor,27386,0.021145,1.509382,7.568877,0.010008,0.075531,0.997093,2025-03-24 14:47:19.819000,P0,1,0.025,0.031003,1.240133,True
2,1742842039847,27.43442,1273,CenterEyeAnchor,27386,0.018981,1.507867,7.599664,0.000239,0.062762,0.998028,2025-03-24 14:47:19.847000,P0,1,0.028,0.030863,1.102249,True
3,1742842039874,27.46282,1274,CenterEyeAnchor,27386,0.016246,1.505601,7.630500,-0.007303,0.056803,0.998359,2025-03-24 14:47:19.874000,P0,1,0.027,0.030957,1.146557,True
4,1742842039902,27.48971,1275,CenterEyeAnchor,27386,0.013274,1.502432,7.661299,-0.012210,0.057715,0.998258,2025-03-24 14:47:19.902000,P0,1,0.028,0.030942,1.105074,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85462,1743103505745,257.28380,13178,CenterEyeAnchor,27386,-0.230588,1.543497,7.376041,0.368187,0.004125,0.929743,2025-03-27 15:25:05.745000,P6,8,0.028,0.040468,1.445291,True
85463,1743103505773,257.31080,13179,CenterEyeAnchor,27386,-0.232035,1.554049,7.418886,0.398355,0.011807,0.917155,2025-03-27 15:25:05.773000,P6,8,0.028,0.042869,1.531051,True
85464,1743103505801,257.33880,13180,CenterEyeAnchor,27386,-0.230375,1.565750,7.459814,0.417578,0.033772,0.908013,2025-03-27 15:25:05.801000,P6,8,0.028,0.040962,1.462916,True
85465,1743103505829,257.36710,13181,CenterEyeAnchor,27386,-0.230330,1.571815,7.499619,0.440730,0.026427,0.897251,2025-03-27 15:25:05.829000,P6,8,0.028,0.039805,1.421608,True


,start_unix_ms,start_frame,start_rel_timestamp,start_local_timestamp,end_unix_ms,end_frame,end_rel_timestamp,end_local_timestamp,participant,trial_number
0,1742842039794,1271,27.37964,2025-03-24 14:47:19.794000,1742842039930,1276,27.51798,2025-03-24 14:47:19.930000,P0,1
1,1742842056625,2286,44.22340,2025-03-24 14:47:36.625000,1742842061770,2631,49.37091,2025-03-24 14:47:41.770000,P0,1
2,1742842091492,4003,79.09109,2025-03-24 14:48:11.492000,1742842095020,4174,82.61163,2025-03-24 14:48:15.020000,P0,2
3,1742842109121,4985,96.72047,2025-03-24 14:48:29.121000,1742842114119,5327,101.71970,2025-03-24 14:48:34.119000,P0,3
4,1742842122819,5773,110.41890,2025-03-24 14:48:42.819000,1742842124191,5844,111.79080,2025-03-24 14:48:44.191000,P0,4
...,...,...,...,...,...,...,...,...,...,...
62,1743103410756,7912,162.30430,2025-03-27 15:23:30.756000,1743103414427,8058,165.96510,2025-03-27 15:23:34.427000,P6,4
63,1743103444649,9849,196.19810,2025-03-27 15:24:04.649000,1743103448781,10129,200.33120,2025-03-27 15:24:08.781000,P6,5
64,1743103475001,11567,226.55380,2025-03-27 15:24:35.001000,1743103478167,11683,229.70530,2025-03-27 15:24:38.167000,P6,6
65,1743103484367,12037,235.91550,2025-03-27 15:24:44.367000,1743103487220,12229,238.77040,2025-03-27 15:24:47.220000,P6,7


# Eye Data

Honestly, the eye data should be the easiest. Since the newest file type, `eye_target.csv`, contains the gaze distance data already, all we need to do is create a new column for the local timestamp and concatenate all user files together.

In [20]:
# Initialize the eye dfs list, which we'll concatenate later
eyes_dfs = []

# Loop through each participant. This time, we loop through indices.
# This allows us to access the data in `positions_dfs`, `players_dfs`, and `on_roads_dfs`
for i in range(len(_PARTICIPANTS)):
    # Get the neceesary files, check if file exists or not
    p = _PARTICIPANTS[i]
    pdir = os.path.join(_INPUT_DATA_DIR, p)
    pfiles = get_immediate_files(pdir)
    if "eye_targets.csv" not in pfiles:
        print(f"ERROR: 'eye_target.csv' not available in {p}'s data'")
        continue
    
    # We already have trial data from our previous loop
    trial_df = trials_dfs[i]
    start_frame = trial_df['start_frame'].iloc[0]
    end_frame = trial_df['end_frame'].iloc[-1]
    trial_intervals = pd.IntervalIndex.from_arrays(trial_df['start_frame'], trial_df['end_frame'], closed='left')
    
    # We already have our crossing data from our previous loop
    crossing_df = on_roads_dfs[i]
    crossing_intervals = pd.IntervalIndex.from_arrays(crossing_df['start_frame'], crossing_df['end_frame'], closed='both')
    
    # read the csv file for the eye data. Filter by trial start and end frames
    eye_df = pd.read_csv(os.path.join(pdir,'eye_targets.csv'))
    eye_df = eye_df[eye_df['frame'].between(start_frame, end_frame)]
    
    # ======
    # We want to process each row's `raycast_hit_name` and generate new info from them.
    # =====
    
    # 1. Replace NaN with ''
    eye_df['raycast_hit_name'] = eye_df['raycast_hit_name'].convert_dtypes().fillna('')
    
    # 2. Create new columns to parse `raycast_hit_name`. This is done by splitting `raycast_hit_name` 
    #    into different subdivisions, based on some delimiters in the name. We need a helper function 
    #.   `raycast_processing()` for this.
    eye_df[['raycast_target_category','raycast_target_type','raycast_target_subtype','raycast_target_name']] = eye_df.apply(
        lambda r: raycast_processing(r['raycast_hit_name']), 
        axis='columns', 
        result_type='expand'
    )
    
    # =====
    # For vehicles, we need additional processing
    # ====
    
    # 1. replace erroneous labels
    eye_df.loc[eye_df['raycast_target_category'] == 'Vehicel', 'raycast_target_category'] = 'Vehicle'
    
    # 2. For vehicle rows, switch the raycast target type with subtype
    vehicle_mask = eye_df['raycast_target_category'] == 'Vehicle'
    eye_df.loc[vehicle_mask, ['raycast_target_type', 'raycast_target_subtype']] = eye_df.loc[vehicle_mask, ['raycast_target_subtype', 'raycast_target_type']].values

    # We want to add columns for local timestamp, participant, trial_number, and whether the user is crossing
    eye_df['local_timestamp'] = eye_df['unix_ms'].apply(lambda t: datetime.datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))
    eye_df['participant'] = p
    tidx = trial_intervals.get_indexer(eye_df['frame'])
    eye_df['trial_number'] = trial_df['trial_number'].iloc[tidx].where(tidx != -1).values
    eye_df['on_road'] = eye_df['frame'].apply(lambda frame: any(frame in interval for interval in crossing_intervals))
    
    # save our results to eyes_dfs
    eyes_dfs.append(eye_df)

In [21]:
# Concatenate and save our eye data
# Let's concatenate everything together into their massive dataframes
eyes_df = pd.concat(eyes_dfs, ignore_index=True)
eyes_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'eyes_all.csv'))

display(eyes_df)

,unix_ms,rel_timestamp,frame,event,side,screen_pos_x,screen_pos_y,screen_pos_z,target_name,local_direction_x,...,raycast_hit_name,target_distance,raycast_target_category,raycast_target_type,raycast_target_subtype,raycast_target_name,local_timestamp,participant,trial_number,on_road
0,1742842039793,23.55420,1271,Eye Hit,Center,698.8971,587.2340,47.90478,NorthSidewalk,-1.413949,...,Environment.Buildings.North-North.Buildings,6.383855,Environment,Buildings,North,North.Buildings,2025-03-24 14:47:19.793000,P0,1,True
1,1742842039817,23.58426,1272,Eye Hit,Center,704.3360,594.4648,48.04549,NorthSidewalk,-1.052613,...,Environment.Buildings.North-North.Buildings,6.350557,Environment,Buildings,North,North.Buildings,2025-03-24 14:47:19.817000,P0,1,True
2,1742842039846,23.60898,1273,Eye Hit,Center,706.9957,598.0370,48.11570,NorthSidewalk,-0.875158,...,Environment.Buildings.North-North.Buildings,6.330110,Environment,Buildings,North,North.Buildings,2025-03-24 14:47:19.846000,P0,1,True
3,1742842039873,23.63737,1274,Eye Hit,Center,709.3895,599.3560,48.14617,NorthSidewalk,-0.714512,...,Environment.Buildings.North-North.Buildings,6.304724,Environment,Buildings,North,North.Buildings,2025-03-24 14:47:19.873000,P0,1,True
4,1742842039901,23.66427,1275,Eye Hit,Center,712.4354,598.7286,48.10818,NorthSidewalk,-0.508997,...,Environment.Buildings.North-North.Buildings,6.273238,Environment,Buildings,North,North.Buildings,2025-03-24 14:47:19.901000,P0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57533,1743103505743,253.62810,13178,NaN,Center,874.9861,748.9743,48.62334,NaN,10.540240,...,Environment.Buildings.North-North.Buildings,7.745037,Environment,Buildings,North,North.Buildings,2025-03-27 15:25:05.743000,P6,8,True
57534,1743103505771,253.65510,13179,NaN,Center,849.7216,733.7180,48.88762,NaN,8.870008,...,Environment.Buildings.North-North.Buildings,7.692665,Environment,Buildings,North,North.Buildings,2025-03-27 15:25:05.771000,P6,8,True
57535,1743103505800,253.68310,13180,NaN,Center,857.0637,714.2510,48.68610,NaN,9.333412,...,Environment.Buildings.North-North.Buildings,7.810331,Environment,Buildings,North,North.Buildings,2025-03-27 15:25:05.800000,P6,8,True
57536,1743103505828,253.71140,13181,NaN,Center,848.4702,721.7515,48.84898,NaN,8.777498,...,Environment.Buildings.North-North.Buildings,7.839416,Environment,Buildings,North,North.Buildings,2025-03-27 15:25:05.828000,P6,8,True


# EEG Analysis

Processing the EEG data is the real difficult task. For the EEG data, we have to:

1. Normalize the data to their resting states
2. FIlter all rows where AF7, AF8, TP9, and TP10 are static for at least 3 rows, per column
3. Add local timestamps, participants, etc. to the dataframes
4. Conduct a separate spearman correlation on all possible elements.

## Eye -> Gazes

In [29]:
# Initialize a list of gazes per participant
gazes_dfs = []

# Initialize the list of columns we want to keep when convert from eyes to gazes
keep_rows = [
    'unix_ms','rel_timestamp','frame','local_timestamp',
    'participant','trial_number','on_road', 'target_distance',
    'raycast_target_category','raycast_target_type',
    'raycast_target_subtype','raycast_target_name'
]

# Iterate through each participant
for i in range(len(_PARTICIPANTS)):
    # Get our participant data
    p = _PARTICIPANTS[i]
    pdir = os.path.join(_INPUT_DATA_DIR, p)
    
    # get our eye data
    eye_df = eyes_dfs[i]
    #display(eye_df)

    # We want to generate Gaze data from Eye data.
    # The distinction is that gazes will be grouped by consecutive raycast hit names and will have a start and end timestamp
    
    # Step 1: Group by `raycast_hit_name`. 
    #.        We keep some required columns, such as `unix_ms` and `rel_timestamp`, `frame`, and `local_timestamp`.
    grouped_df = eye_df.groupby('raycast_hit_name', sort=False).first()[keep_rows]
    gazes_df = grouped_df.rename(columns={
        'unix_ms': 'start_unix_ms',
        'rel_timestamp': 'start_rel_timestamp',
        'frame': 'start_frame',
        'local_timestamp':'start_local_timestamp'
    }).reset_index()
    
    # Step 2: Calculate end timestamps and duration timestamps
    # Since we labeled each Gaze group as the start timestamps, we need to declare the end 
    # timestamp equivalents. We simply use the next Gaze group's start timestamps.
    gazes_df['end_unix_ms'] = gazes_df['start_unix_ms'].shift(-1)
    gazes_df['end_rel_timestamp'] = gazes_df['start_rel_timestamp'].shift(-1)
    gazes_df['end_frame'] = gazes_df['start_frame'].shift(-1)
    gazes_df['end_local_timestamp'] = gazes_df['start_local_timestamp'].shift(-1)
    # For the last Gaze, since there's no latter Gaze, we just get the last known row of the DataFrame
    last_row = eye_df.iloc[-1]
    gazes_df.iloc[-1, gazes_df.columns.get_loc('end_unix_ms')] = last_row['unix_ms']
    gazes_df.iloc[-1, gazes_df.columns.get_loc('end_rel_timestamp')] = last_row['rel_timestamp']
    gazes_df.iloc[-1, gazes_df.columns.get_loc('end_frame')] = last_row['frame']
    gazes_df.iloc[-1, gazes_df.columns.get_loc('end_local_timestamp')] = last_row['local_timestamp']
    # We now calculate their durations
    gazes_df['duration_frame'] = gazes_df['end_frame'] - gazes_df['start_frame']
    gazes_df['duration_unix_ms'] = gazes_df['end_unix_ms'] - gazes_df['start_unix_ms']
    gazes_df['duration_rel_timestamp'] = gazes_df['end_rel_timestamp'] - gazes_df['start_rel_timestamp']
    
    # With this data, we can safely cache it
    gazes_dfs.append(gazes_df)    

In [30]:
# Concatenate and save our gaze data
gazes_df = pd.concat(gazes_dfs, ignore_index=True)
gazes_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'gazes_all.csv'))

display(gazes_df)

,raycast_hit_name,start_unix_ms,start_rel_timestamp,start_frame,start_local_timestamp,participant,trial_number,on_road,target_distance,raycast_target_category,raycast_target_type,raycast_target_subtype,raycast_target_name,end_unix_ms,end_rel_timestamp,end_frame,end_local_timestamp,duration_frame,duration_unix_ms,duration_rel_timestamp
0,Environment.Buildings.North-North.Buildings,1742842039793,23.55420,1271,2025-03-24 14:47:19.793000,P0,1,True,6.383855,Environment,Buildings,North,North.Buildings,1.742842e+12,24.22255,1295.0,2025-03-24 14:47:20.458000,24.0,665.0,0.66835
1,Environment.Sidewalk.North-North.Sidewalk,1742842040458,24.22255,1295,2025-03-24 14:47:20.458000,P0,1,False,5.680598,Environment,Sidewalk,North,North.Sidewalk,1.742842e+12,24.84619,1318.0,2025-03-24 14:47:21.074000,23.0,616.0,0.62364
2,Environment.Road.West-Road.West,1742842041074,24.84619,1318,2025-03-24 14:47:21.074000,P0,1,False,35.174800,Environment,Road,West,Road.West,1.742842e+12,24.89638,1321.0,2025-03-24 14:47:21.123000,3.0,49.0,0.05019
3,Vehicle.Jeep.Body-Jeep5.Body,1742842041123,24.89638,1321,2025-03-24 14:47:21.123000,P0,1,False,23.095630,Vehicle,Body,Jeep,Jeep5.Body,1.742842e+12,24.94458,1324.0,2025-03-24 14:47:21.170000,3.0,47.0,0.04820
4,Vehicle.Jeep.Wheel-Jeep5.Wheel.BR,1742842041170,24.94458,1324,2025-03-24 14:47:21.170000,P0,1,False,25.192560,Vehicle,Wheel,Jeep,Jeep5.Wheel.BR,1.742842e+12,24.97499,1326.0,2025-03-24 14:47:21.201000,2.0,31.0,0.03041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,Vehicle.Car.Wheel-Car5.Wheel.FR,1743103490766,238.65860,12422,2025-03-27 15:24:50.766000,P6,8,False,28.715640,Vehicle,Wheel,Car,Car5.Wheel.FR,1.743103e+12,239.19240,12454.0,2025-03-27 15:24:51.303000,32.0,537.0,0.53380
934,Vehicel.Car.Door-Car5.Door.FR,1743103491303,239.19240,12454,2025-03-27 15:24:51.303000,P6,8,False,25.839860,Vehicle,Door,Car,Car5.Door.FR,1.743103e+12,240.20050,12515.0,2025-03-27 15:24:52.306000,61.0,1003.0,1.00810
935,Vehicle.Sedan.Door-Sedan4.Door.BR,1743103492306,240.20050,12515,2025-03-27 15:24:52.306000,P6,8,False,27.758710,Vehicle,Door,Sedan,Sedan4.Door.BR,1.743103e+12,240.24760,12518.0,2025-03-27 15:24:52.353000,3.0,47.0,0.04710
936,Vehicle.Sedan.Door-Sedan4.Door.FR,1743103492353,240.24760,12518,2025-03-27 15:24:52.353000,P6,8,False,26.120060,Vehicle,Door,Sedan,Sedan4.Door.FR,1.743103e+12,241.11090,12570.0,2025-03-27 15:24:53.218000,52.0,865.0,0.86330


## EEG Pre-Processing, Crossing Offset Calc.

EEG analysis is dependent on gaze data. The first thing we need to do is to filter the EEG itself.
We need a helper function that drops useless columns and filters rows, while also converting each power into their base 10 forms.

In [107]:
# Converts the format of a local timestamp into unix seconds. Requires the datetime package
def timestamp_to_unix_seconds(x):
    date_format = datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
    unix_seconds = datetime.datetime.timestamp(date_format)
    return unix_seconds

# Converts the format of a local timestamp into unix milliseconds. 
# Requires the datatime package, and relies on `timestamp_to_unix_seconds()` function.
def timestamp_to_unix_milliseconds(x):
    unix_seconds = timestamp_to_unix_seconds(x)
    unix_milliseconds = int(unix_seconds * 1000)
    return unix_milliseconds

# EEG-specific Constants
_FREQUENCIES = ['Delta','Theta','Alpha','Beta','Gamma']
_FREQUENCY_COLORS = ['purple','red', 'blue', 'green', 'orange']
_ELECTRODES = ['AF7','AF8']
_REST_EEG_TIME_BOUNDS = (10000, 40000)

# Generate freq-band combination column names
freq_band_colnames = []
freq_band_all_colnames = []
for freq in _FREQUENCIES: 
    for electrode in _ELECTRODES:
        freq_band_colnames.append(f'{freq}_{electrode}')
        freq_band_all_colnames.append(f'{freq}_{electrode}')
        freq_band_all_colnames.append(f'{freq}_{electrode}_normalized')

In [108]:
def LoadEEG(src_filepath:str, drop_raw:bool=True, drop_accel:bool=True, drop_gyro:bool=True):
    # Read the CSV data
    df = pd.read_csv(src_filepath)
    
    # Remove rows with empty timestamp or battery values.
    df = df[~df['TimeStamp'].isna()]
    df = df[~df['Battery'].isna()]
    
    # Calculate unix milliseconds, given the localized timestamp.
    df['unix_ms'] = df['TimeStamp'].apply(lambda x: int(timestamp_to_unix_milliseconds(x)))

    # Drop useless columns
    drop_list = ['TimeStamp', 'Elements', 'Battery', 'HeadBandOn','AUX_RIGHT','AUX_LEFT','HSI_TP9','HSI_AF7','HSI_AF8','HSI_TP10']
    if drop_raw: drop_list.extend(['RAW_AF7','RAW_AF8','RAW_TP9','RAW_TP10'])
    if drop_accel: drop_list.extend(['Accelerometer_X','Accelerometer_Y','Accelerometer_Z'])
    if drop_gyro: drop_list.extend(['Gyro_X','Gyro_Y','Gyro_Z'])
    df.drop(columns=drop_list, inplace=True) # Drop useless columns
    
    # For each frequency, exit from log to actual power, then remove the older column
    for freq in _FREQUENCIES:
        for electrode in _ELECTRODES:
            in_colname = f'{freq}_{electrode}'
            out_colname = f'{in_colname}_Pow'
            df[out_colname] = 10 ** df[in_colname]
            df.drop(columns=[in_colname], inplace=True)
            df.rename(columns={out_colname:in_colname},inplace=True)
    
    # Return the EEG data
    return df

In [109]:
def FilterConsecutiveRows(df, columns_to_check, num_samples_bound=3):
    
    # Initialize a mask of rows to remove
    remove_mask = pd.Series(False, index=df.index)
    
    for col in columns_to_check:
        # Find groups of consecutive same values
        group_id = (df[col] != df[col].shift()).cumsum()
    
        # Group by those consecutive runs
        grouped = df.groupby(group_id)

        for _, idxs in grouped.groups.items():
            idxs = list(idxs)
            # Check if group has same value and length >= 3
            if len(idxs) >= num_samples_bound:
                val = df.loc[idxs[0], col]
                if all(df.loc[i, col] == val for i in idxs):
                    # Mark all but the first for removal
                    remove_mask[idxs[1:]] = True

    # Apply mask to filter
    filtered_df = df[~remove_mask].reset_index(drop=True)
    
    # Return
    return filtered_df

In [110]:
def CalculateOffsets(samples_df, sessions_df, group_colname, sample_timestamp_colname, session_start_colname, out_colname):
    
    results = []

    # Grouping by 'group'
    for group, sample_group in samples_df.groupby(group_colname):
        trial_group = sessions_df[sessions_df[group_colname] == group].sort_values(session_start_colname)
        trial_starts = trial_group[session_start_colname].values
        sample_timestamps = sample_group[sample_timestamp_colname].values

        # Initialize output
        deltas = []

        for ts in sample_timestamps:
            # Find first trial start AFTER this timestamp
            future_starts = trial_starts[trial_starts > ts]
            if len(future_starts) > 0:
                next_start = future_starts[0]
            else:
                next_start = trial_starts[-1]  # Fallback to latest trial start
            deltas.append(ts - next_start)

        # Append results for this group
        sample_group = sample_group.copy()
        sample_group[out_colname] = deltas
        results.append(sample_group)

    # Final DataFrame with the new column
    final_df = pd.concat(results).sort_index()
    display(final_df)
    return final_df[out_colname]

In [120]:
# initialize the eeg dataframe lists for the raw eeg and filtered eeg
eegs_raws_dfs = []
eegs_filtereds_dfs = []

# iterate through our pedestrians
for i in range(len(_PARTICIPANTS)):
    p = _PARTICIPANTS[i]
    pdir = os.path.join(_INPUT_DATA_DIR, p)
    pfiles = get_immediate_files(pdir)
    if "eeg_rest.csv" not in pfiles:
        print(f"ERROR: 'eeg_rest.csv' not available in {p}'s data'")
        continue
    if "eeg_vr.csv" not in pfiles:
        print(f"ERROR: 'eeg_vr.csv' not available in {p}'s data'")
        continue
    
    # Get trial and crossing df
    trial_df = trials_dfs[i]
    start_unix = trial_df['start_unix'].iloc[0]
    end_unix = trial_df['end_unix'].iloc[-1]
    trial_intervals = pd.IntervalIndex.from_arrays(trial_df['start_unix'], trial_df['end_unix'], closed='left')
    crossing_df = on_roads_dfs[i]
    
    # Read the rest eeg data. We make sure to filter the rows to 30 seconds of the trial.
    rest_eeg_df = LoadEEG(os.path.join(pdir, 'eeg_rest.csv'))
    #rest_eeg_df['local_timestamp'] = rest_eeg_df['unix_ms'].apply(lambda t: datetime.datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))
    rest_eeg_df['rel_unix_ms'] = rest_eeg_df['unix_ms'] - rest_eeg_df['unix_ms'].iloc[0]
    rest_eeg_df = rest_eeg_df[
        (rest_eeg_df['rel_unix_ms']>= _REST_EEG_TIME_BOUNDS[0]) 
        & (rest_eeg_df['rel_unix_ms']<= _REST_EEG_TIME_BOUNDS[1])
    ]
    
    # Read the trial EEG data. We do NOT filter them by time, but we still normalize the data
    eeg_df = LoadEEG(os.path.join(pdir, 'eeg_vr.csv'))
    for freq_band in freq_band_colnames:
        # read the min and max of the provided frequency band
        freq_band_min = rest_eeg_df[freq_band].min()
        freq_band_max = rest_eeg_df[freq_band].max()
        eeg_df[f'{freq_band}_normalized'] = (eeg_df[freq_band] - freq_band_min) / (freq_band_max - freq_band_min)
    
    # For now, let's append some columns to EEG
    eeg_df = eeg_df[eeg_df['unix_ms'].between(start_unix, end_unix)]
    eeg_df['local_timestamp'] = eeg_df['unix_ms'].apply(lambda t: datetime.datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))
    eeg_df['participant'] = p
    tidx = trial_intervals.get_indexer(eeg_df['unix_ms'])
    eeg_df['trial_number'] = trial_df['trial_number'].iloc[tidx].where(tidx != -1).values
    crossing_intervals = pd.IntervalIndex.from_arrays(crossing_df['start_unix_ms'], crossing_df['end_unix_ms'], closed='both')
    eeg_df['on_road'] = eeg_df['unix_ms'].apply(lambda ts: any(ts in interval for interval in crossing_intervals))
    eeg_df['crossing_offset_unix_ms'] = CalculateOffsets(
        eeg_df, 
        crossing_df, 
        group_colname='trial_number', 
        sample_timestamp_colname = 'unix_ms',
        session_start_colname = 'start_unix_ms',
        out_colname = 'crossing_offset_unix_ms'
    )
    
    # Now, we conduct a filter to restrict the rows themselves
    eeg_filtered_df = FilterConsecutiveRows(eeg_df, freq_band_colnames, num_samples_bound=5)
    
    # Don't add anything just yet. Instead, just add this to `eegs_filtered_df`. We'll process them later.
    eegs_raws_dfs.append(eeg_df)
    eegs_filtereds_dfs.append(eeg_filtered_df)
    display(eeg_filtered_df)


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
143,1.258997,1.414490,1.312129,0.771797,0.826205,0.902003,0.494335,0.786802,0.113748,0.843657,...,1.243803,4.959159,0.898815,8.049836,0.899327,2025-03-24 14:47:19.957000,P0,1,False,-16668
144,1.368466,1.414490,1.367782,0.771797,1.005129,0.902003,0.560295,0.786802,0.058877,0.843657,...,1.178277,4.003876,0.945980,6.081807,0.977698,2025-03-24 14:47:20.462000,P0,1,False,-16163
146,1.597498,1.414490,1.435598,0.771797,0.989920,0.902003,0.604796,0.786802,0.009563,0.843657,...,1.428076,4.132484,0.676362,5.573786,1.176875,2025-03-24 14:47:20.956000,P0,1,False,-15669
147,1.745149,1.414490,1.486255,0.771797,0.972852,0.902003,0.591050,0.786802,-0.142719,0.843657,...,1.276395,4.450017,0.694479,5.308864,1.629101,2025-03-24 14:47:21.457000,P0,1,False,-15168
148,1.746012,1.414490,1.238417,0.771797,0.885606,0.902003,0.509056,0.786802,0.166789,0.843657,...,1.132252,5.135644,0.743673,5.590303,1.171631,2025-03-24 14:47:21.957000,P0,1,False,-14668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,1.576011,1.043644,1.104711,0.995352,0.794780,0.961304,0.867334,0.571313,0.642557,0.143217,...,0.411319,13.507472,3.956536,33.159857,6.986894,2025-03-24 14:50:19.458000,P0,8,True,6466
585,1.576011,1.043644,1.104711,0.995352,0.794780,0.961304,0.867334,0.571313,0.642557,0.143217,...,0.411319,13.507472,3.956536,33.159857,6.986894,2025-03-24 14:50:19.958000,P0,8,True,6966
586,1.576011,1.043644,1.104711,0.995352,0.794780,0.961304,0.867334,0.571313,0.642557,0.143217,...,0.411319,13.507472,3.956536,33.159857,6.986894,2025-03-24 14:50:20.458000,P0,8,True,7466
587,1.576011,1.043644,1.104711,0.995352,0.794780,0.961304,0.867334,0.571313,0.642557,0.143217,...,0.411319,13.507472,3.956536,33.159857,6.986894,2025-03-24 14:50:20.958000,P0,8,True,7966


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.258997,1.414490,1.312129,0.771797,0.826205,0.902003,0.494335,0.786802,0.113748,0.843657,...,1.243803,4.959159,0.898815,8.049836,0.899327,2025-03-24 14:47:19.957000,P0,1,False,-16668
1,1.368466,1.414490,1.367782,0.771797,1.005129,0.902003,0.560295,0.786802,0.058877,0.843657,...,1.178277,4.003876,0.945980,6.081807,0.977698,2025-03-24 14:47:20.462000,P0,1,False,-16163
2,1.597498,1.414490,1.435598,0.771797,0.989920,0.902003,0.604796,0.786802,0.009563,0.843657,...,1.428076,4.132484,0.676362,5.573786,1.176875,2025-03-24 14:47:20.956000,P0,1,False,-15669
3,1.745149,1.414490,1.486255,0.771797,0.972852,0.902003,0.591050,0.786802,-0.142719,0.843657,...,1.276395,4.450017,0.694479,5.308864,1.629101,2025-03-24 14:47:21.457000,P0,1,False,-15168
4,1.746012,1.414490,1.238417,0.771797,0.885606,0.902003,0.509056,0.786802,0.166789,0.843657,...,1.132252,5.135644,0.743673,5.590303,1.171631,2025-03-24 14:47:21.957000,P0,1,False,-14668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,0.948085,1.043644,0.877619,0.995352,1.120057,0.961304,0.819338,0.571313,0.500121,0.143217,...,0.937016,4.545605,4.634735,12.718544,7.666299,2025-03-24 14:50:16.958000,P0,8,True,3966
143,1.003095,1.043644,0.870132,0.995352,1.121656,0.961304,0.857986,0.571313,0.551424,0.143217,...,0.680988,7.540045,3.913909,19.858220,6.740146,2025-03-24 14:50:17.469000,P0,8,True,4477
144,1.431165,1.043644,1.009312,0.995352,0.807137,0.961304,0.809885,0.571313,0.551955,0.143217,...,0.420873,9.669001,3.331865,26.809981,7.182419,2025-03-24 14:50:17.967000,P0,8,True,4975
145,1.576011,1.043644,1.104711,0.995352,0.794780,0.961304,0.867334,0.571313,0.642557,0.143217,...,0.411787,13.507472,3.860435,33.159857,7.370416,2025-03-24 14:50:18.469000,P0,8,True,5477


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
87,1.366736,1.542858,1.074675,1.424260,0.861299,1.379777,0.582147,0.750607,0.260837,0.264770,...,6.549447,0.565447,1.297674,-0.089315,0.570587,2025-03-24 15:35:21.753000,P1,1,False,-13802
88,1.366736,1.514174,1.074675,1.439539,0.861299,1.319792,0.582147,0.712024,0.260837,0.391919,...,3.468387,0.542492,1.162890,-0.006784,0.284377,2025-03-24 15:35:22.253000,P1,1,False,-13302
89,1.366736,1.338750,1.074675,1.380759,0.861299,1.140598,0.582147,0.649213,0.260837,0.451724,...,4.348144,0.549239,1.269357,-0.037098,0.657135,2025-03-24 15:35:22.752000,P1,1,False,-12803
90,1.366736,1.023143,1.074675,1.176435,0.861299,1.132102,0.582147,0.570533,0.260837,0.484693,...,3.745694,0.507466,1.222264,-0.229657,0.649761,2025-03-24 15:35:23.261000,P1,1,False,-12294
91,1.366736,0.722191,1.074675,0.678832,0.861299,1.095476,0.582147,0.428500,0.260837,0.336490,...,3.112027,0.622344,1.078753,-0.196904,0.611409,2025-03-24 15:35:23.752000,P1,1,False,-11803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.969305,1.189626,1.027383,-0.152560,0.584126,2025-03-24 15:38:07.759000,P1,8,True,1924
539,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.552970,1.101361,0.786770,-0.200504,0.749000,2025-03-24 15:38:08.263000,P1,8,True,2428
540,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.590015,1.169600,0.728919,-0.214873,0.853664,2025-03-24 15:38:08.754000,P1,8,True,2919
543,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.334241,0.843064,0.627901,-0.128986,0.655404,2025-03-24 15:38:09.254000,P1,8,True,3419


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.366736,1.542858,1.074675,1.424260,0.861299,1.379777,0.582147,0.750607,0.260837,0.264770,...,6.549447,0.565447,1.297674,-0.089315,0.570587,2025-03-24 15:35:21.753000,P1,1,False,-13802
1,1.366736,1.514174,1.074675,1.439539,0.861299,1.319792,0.582147,0.712024,0.260837,0.391919,...,3.468387,0.542492,1.162890,-0.006784,0.284377,2025-03-24 15:35:22.253000,P1,1,False,-13302
2,1.366736,1.338750,1.074675,1.380759,0.861299,1.140598,0.582147,0.649213,0.260837,0.451724,...,4.348144,0.549239,1.269357,-0.037098,0.657135,2025-03-24 15:35:22.752000,P1,1,False,-12803
3,1.366736,1.023143,1.074675,1.176435,0.861299,1.132102,0.582147,0.570533,0.260837,0.484693,...,3.745694,0.507466,1.222264,-0.229657,0.649761,2025-03-24 15:35:23.261000,P1,1,False,-12294
4,1.366736,0.722191,1.074675,0.678832,0.861299,1.095476,0.582147,0.428500,0.260837,0.336490,...,3.112027,0.622344,1.078753,-0.196904,0.611409,2025-03-24 15:35:23.752000,P1,1,False,-11803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.969305,1.189626,1.027383,-0.152560,0.584126,2025-03-24 15:38:07.759000,P1,8,True,1924
323,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.552970,1.101361,0.786770,-0.200504,0.749000,2025-03-24 15:38:08.263000,P1,8,True,2428
324,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.590015,1.169600,0.728919,-0.214873,0.853664,2025-03-24 15:38:08.754000,P1,8,True,2919
325,1.354568,1.520990,1.162398,1.603589,1.398080,1.236847,0.628221,1.011554,0.119297,0.524295,...,1.334241,0.843064,0.627901,-0.128986,0.655404,2025-03-24 15:38:09.254000,P1,8,True,3419


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
132,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.22378,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:25.068000,P2,1,True,-21868
135,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.22378,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:25.569000,P2,1,False,-21367
137,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.22378,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:26.075000,P2,1,False,-20861
138,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.22378,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:26.569000,P2,1,False,-20367
139,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.22378,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:27.069000,P2,1,False,-19867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,1.129232,0.735225,1.971809,0.384581,1.401636,0.708349,1.447891,1.162243,1.29372,0.988048,...,0.494607,6.805063,6.915184,5.288377,9.478860,2025-03-25 16:08:33.574000,P2,8,True,1783
672,1.129232,0.735225,1.971809,0.384581,1.401636,0.708349,1.447891,1.162243,1.29372,0.988048,...,0.494607,6.805063,6.915184,5.288377,9.478860,2025-03-25 16:08:34.071000,P2,8,True,2280
673,1.129232,0.735225,1.971809,0.384581,1.401636,0.708349,1.447891,1.162243,1.29372,0.988048,...,0.494607,6.805063,6.915184,5.288377,9.478860,2025-03-25 16:08:34.582000,P2,8,True,2791
675,1.129232,0.735225,1.971809,0.384581,1.401636,0.708349,1.447891,1.162243,1.29372,0.988048,...,0.494607,6.805063,6.915184,5.288377,9.478860,2025-03-25 16:08:35.077000,P2,8,True,3286


...

,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,1.382039,26.085369,22.519255,20.603089,38.857508,2025-03-26 11:37:05.126000,P4,1,True,-38135
1,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.405541,30.168133,19.084177,25.350394,33.906579,2025-03-26 11:37:05.864000,P4,1,False,-37397
2,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.574645,31.821393,18.980858,23.570033,33.353280,2025-03-26 11:37:05.930000,P4,1,False,-37331
3,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.616261,33.352428,18.859268,21.927867,33.158481,2025-03-26 11:37:06.026000,P4,1,False,-37235
4,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.627268,33.498252,19.048664,21.076937,33.574622,2025-03-26 11:37:06.147000,P4,1,False,-37114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.492015,0.691901,0.162093,0.501511,0.635849,0.735100,1.436978,1.311504,1.175309,1.023283,...,2.320303,35.455100,26.370170,49.107983,26.153355,2025-03-26 11:42:12.031000,P4,8,False,-7574
121,0.467749,0.674509,0.181663,0.518996,0.635849,0.735100,1.457984,1.334894,1.193623,1.038244,...,2.225404,34.679375,25.981561,48.055424,27.123099,2025-03-26 11:42:12.162000,P4,8,False,-7443
122,0.422509,0.616503,0.185518,0.510978,0.635849,0.723368,1.477326,1.358018,1.215090,1.047363,...,2.097348,32.898485,25.803229,46.753897,28.163097,2025-03-26 11:42:12.254000,P4,8,False,-7351
123,-0.035763,0.181935,0.068337,0.150384,0.472057,0.609622,1.492433,1.338895,1.246645,1.064089,...,1.458690,30.853909,18.549090,48.027948,31.349538,2025-03-26 11:42:12.767000,P4,8,False,-6838


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
18466,1.226269,1.444629,1.14706,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.307000,P5,1,True,-6598
18467,1.226269,1.444629,1.14706,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.308000,P5,1,True,-6597
18468,1.226269,1.444629,1.14706,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.308000,P5,1,True,-6597
18469,1.226269,1.444629,1.14706,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.309000,P5,1,True,-6596
18470,1.226269,1.444629,1.14706,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.310000,P5,1,True,-6595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68448,1.290524,1.651413,1.22557,2.000962,0.830894,1.666763,0.680091,1.006410,0.181997,0.425032,...,0.004187,0.007538,0.010490,0.009437,0.017573,2025-03-27 13:06:39.041000,P5,8,False,3738
68449,1.290524,1.651413,1.22557,2.000962,0.830894,1.666763,0.680091,1.006410,0.181997,0.425032,...,0.004187,0.007538,0.010490,0.009437,0.017573,2025-03-27 13:06:39.042000,P5,8,False,3739
68450,1.290524,1.651413,1.22557,2.000962,0.830894,1.666763,0.680091,1.006410,0.181997,0.425032,...,0.004187,0.007538,0.010490,0.009437,0.017573,2025-03-27 13:06:39.042000,P5,8,False,3739
68451,1.290524,1.651413,1.22557,2.000962,0.830894,1.666763,0.680091,1.006410,0.181997,0.425032,...,0.004187,0.007538,0.010490,0.009437,0.017573,2025-03-27 13:06:39.043000,P5,8,False,3740


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.226269,1.444629,1.147060,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.307000,P5,1,True,-6598
1,1.238642,1.484778,1.146012,1.258805,1.235578,1.272569,0.639299,0.594300,-0.020470,-0.022165,...,0.001177,0.004733,-0.000490,0.004060,0.000643,2025-03-27 13:03:24.360000,P5,1,True,-6545
2,1.274606,1.543494,1.161744,1.322076,1.251457,1.314910,0.659406,0.577512,0.022853,-0.003746,...,0.001649,0.005011,-0.000104,0.003828,0.000895,2025-03-27 13:03:24.448000,P5,1,True,-6457
3,1.338565,1.621397,1.207561,1.403015,1.279780,1.375856,0.673247,0.557043,0.060165,0.018488,...,0.002040,0.005217,0.000276,0.003501,0.001336,2025-03-27 13:03:24.567000,P5,1,True,-6338
4,1.485384,1.842649,1.296528,1.751640,1.240559,1.315687,0.338904,0.427362,-0.005953,0.105450,...,0.007796,0.005180,0.002286,0.006563,0.001845,2025-03-27 13:03:26.081000,P5,1,False,-4824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,0.930384,0.464569,0.451023,0.173508,1.098141,0.602302,0.507773,0.386870,0.093398,-0.154948,...,0.004005,0.004058,0.001460,0.004908,-0.002429,2025-03-27 13:06:32.018000,P5,8,False,-3285
535,0.954478,0.533345,0.477180,0.166040,1.092559,0.534318,0.514802,0.374179,0.097631,-0.154399,...,0.003971,0.004193,0.001713,0.005030,-0.002393,2025-03-27 13:06:32.124000,P5,8,False,-3179
536,0.984067,0.647821,0.508948,0.189680,1.092559,0.458577,0.523602,0.361707,0.106810,-0.145752,...,0.003983,0.004384,0.002080,0.005277,-0.002174,2025-03-27 13:06:32.234000,P5,8,False,-3069
537,1.069609,0.770886,0.585373,0.251266,1.092559,0.454215,0.535798,0.356762,0.113261,-0.125068,...,0.004004,0.005061,0.002482,0.006211,-0.001940,2025-03-27 13:06:32.324000,P5,8,False,-2979


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
21433,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,2.240350,2.780085,1.853243,3.294122,2025-03-27 15:21:53.751000,P6,1,True,-22027
21434,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,2.240350,2.780085,1.853243,3.294122,2025-03-27 15:21:53.753000,P6,1,True,-22025
21435,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,2.240350,2.780085,1.853243,3.294122,2025-03-27 15:21:53.755000,P6,1,True,-22023
21436,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,1.414240,2.780085,1.539078,3.294122,2025-03-27 15:21:53.757000,P6,1,True,-22021
21437,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,1.414240,2.780085,1.539078,3.294122,2025-03-27 15:21:53.760000,P6,1,True,-22018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70853,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.864000,P6,8,False,3145
70854,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.866000,P6,8,False,3147
70855,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.866000,P6,8,False,3147
70856,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.867000,P6,8,False,3148


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,2.240350,2.780085,1.853243,3.294122,2025-03-27 15:21:53.751000,P6,1,True,-22027
1,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.172060,1.645343,5.297195,0.842899,3.249684,2025-03-27 15:22:06.741000,P6,1,False,-9037
2,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.357106,1.792102,4.858184,1.254488,4.760463,2025-03-27 15:22:07.265000,P6,1,False,-8513
3,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.405764,1.793515,5.070984,1.293070,4.884799,2025-03-27 15:22:07.340000,P6,1,False,-8438
4,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.419807,1.823281,5.297020,1.296757,4.950804,2025-03-27 15:22:07.426000,P6,1,False,-8352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1.350211,1.082075,0.859904,0.860775,0.624219,1.009369,0.919563,0.882640,0.820015,0.876603,...,0.901483,1.404012,5.805374,1.959626,2.483984,2025-03-27 15:24:52.260000,P6,8,False,-10459
101,0.509189,1.106268,0.402402,0.926340,0.696738,0.910357,0.800595,0.829100,0.765276,0.869278,...,0.193084,3.416110,9.827962,1.482160,1.875070,2025-03-27 15:24:58.055000,P6,8,False,-4664
102,0.485877,0.152995,0.406874,0.098602,0.704198,0.113821,0.790970,0.793800,0.722505,0.963718,...,0.198930,3.190221,9.876969,1.488003,2.445789,2025-03-27 15:24:58.141000,P6,8,False,-4578
103,0.449037,0.121580,0.485228,0.089250,0.747930,0.324655,0.805409,0.793967,0.639590,0.895610,...,0.205011,2.677705,10.558591,1.610461,3.037579,2025-03-27 15:24:58.358000,P6,8,False,-4361


In [122]:
# Concatenate and save our filtered data
eegs_raw_df = pd.concat(eegs_raws_dfs, ignore_index=True)
eegs_raw_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'eegs_raw_all.csv'))

# Concatenate and save our filtered data
eegs_filtered_df = pd.concat(eegs_filtereds_dfs, ignore_index=True)
eegs_filtered_df.to_csv(os.path.join(_OUTPUT_DATA_DIR, 'eegs_filtered_all.csv'))

display(eegs_raw_df)
display(eegs_filtered_df)

,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.258997,1.414490,1.312129,0.771797,0.826205,0.902003,0.494335,0.786802,0.113748,0.843657,...,1.243803,4.959159,0.898815,8.049836,0.899327,2025-03-24 14:47:19.957000,P0,1,False,-16668
1,1.368466,1.414490,1.367782,0.771797,1.005129,0.902003,0.560295,0.786802,0.058877,0.843657,...,1.178277,4.003876,0.945980,6.081807,0.977698,2025-03-24 14:47:20.462000,P0,1,False,-16163
2,1.597498,1.414490,1.435598,0.771797,0.989920,0.902003,0.604796,0.786802,0.009563,0.843657,...,1.428076,4.132484,0.676362,5.573786,1.176875,2025-03-24 14:47:20.956000,P0,1,False,-15669
3,1.745149,1.414490,1.486255,0.771797,0.972852,0.902003,0.591050,0.786802,-0.142719,0.843657,...,1.276395,4.450017,0.694479,5.308864,1.629101,2025-03-24 14:47:21.457000,P0,1,False,-15168
4,1.746012,1.414490,1.238417,0.771797,0.885606,0.902003,0.509056,0.786802,0.166789,0.843657,...,1.132252,5.135644,0.743673,5.590303,1.171631,2025-03-24 14:47:21.957000,P0,1,False,-14668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264891,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.864000,P6,8,False,3145
264892,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.866000,P6,8,False,3147
264893,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.866000,P6,8,False,3147
264894,1.185908,1.050759,0.936466,0.942244,0.685907,1.084133,0.912772,0.792495,0.541220,0.710071,...,0.277584,1.530645,11.744378,2.083967,4.100798,2025-03-27 15:25:05.867000,P6,8,False,3148


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.258997,1.414490,1.312129,0.771797,0.826205,0.902003,0.494335,0.786802,0.113748,0.843657,...,1.243803,4.959159,0.898815,8.049836,0.899327,2025-03-24 14:47:19.957000,P0,1,False,-16668
1,1.368466,1.414490,1.367782,0.771797,1.005129,0.902003,0.560295,0.786802,0.058877,0.843657,...,1.178277,4.003876,0.945980,6.081807,0.977698,2025-03-24 14:47:20.462000,P0,1,False,-16163
2,1.597498,1.414490,1.435598,0.771797,0.989920,0.902003,0.604796,0.786802,0.009563,0.843657,...,1.428076,4.132484,0.676362,5.573786,1.176875,2025-03-24 14:47:20.956000,P0,1,False,-15669
3,1.745149,1.414490,1.486255,0.771797,0.972852,0.902003,0.591050,0.786802,-0.142719,0.843657,...,1.276395,4.450017,0.694479,5.308864,1.629101,2025-03-24 14:47:21.457000,P0,1,False,-15168
4,1.746012,1.414490,1.238417,0.771797,0.885606,0.902003,0.509056,0.786802,0.166789,0.843657,...,1.132252,5.135644,0.743673,5.590303,1.171631,2025-03-24 14:47:21.957000,P0,1,False,-14668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1.350211,1.082075,0.859904,0.860775,0.624219,1.009369,0.919563,0.882640,0.820015,0.876603,...,0.901483,1.404012,5.805374,1.959626,2.483984,2025-03-27 15:24:52.260000,P6,8,False,-10459
1796,0.509189,1.106268,0.402402,0.926340,0.696738,0.910357,0.800595,0.829100,0.765276,0.869278,...,0.193084,3.416110,9.827962,1.482160,1.875070,2025-03-27 15:24:58.055000,P6,8,False,-4664
1797,0.485877,0.152995,0.406874,0.098602,0.704198,0.113821,0.790970,0.793800,0.722505,0.963718,...,0.198930,3.190221,9.876969,1.488003,2.445789,2025-03-27 15:24:58.141000,P6,8,False,-4578
1798,0.449037,0.121580,0.485228,0.089250,0.747930,0.324655,0.805409,0.793967,0.639590,0.895610,...,0.205011,2.677705,10.558591,1.610461,3.037579,2025-03-27 15:24:58.358000,P6,8,False,-4361


In [117]:
# Initialize the finalized eeg dfs list
eegs_dfs = []

for i in range(len(_PARTICIPANTS)):
    # You know the drill...
    p = _PARTICIPANTS[i]
    pdir = os.path.join(_INPUT_DATA_DIR, p)
    
    trial_df = trials_dfs[i]
    start_unix = trial_df['start_unix'].iloc[0]
    end_unix = trial_df['end_unix'].iloc[-1]
    trial_intervals = pd.IntervalIndex.from_arrays(trial_df['start_unix'], trial_df['end_unix'], closed='left')
    
    # Get the filtered EEG dataframe. We'll just refer to it as `eeg_df`. Also load in gazes
    eeg_df = eegs_filtered_dfs[i]
    gazes_df = gazes_dfs[i]
    
    # Let's do some intermittent saving here and 
    
    # The idea now is that for each gaze, we need to calculate several things:
    # 1: the number of samples 
        
    #gaze_intervals = pd.IntervalIndex.from_arrays(gazes_df['start_unix_ms'], gazes_df['end_unix_ms'], closed='left')
    #gidx = gaze_intervals.get_indexer(eeg_df['unix_ms'])
    #eeg_df['gaze_number'] = gazes_df['raycast_target_name'].iloc[gidx].where(gidx != -1).values
    
    #display(gazes_df.head(50))
    
    #crossing_intervals = pd.IntervalIndex.from_arrays(crossing_df['start_frame'], crossing_df['end_frame'], closed='both')
    #display(eeg_df.head(20))
    
    

,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.258997,1.414490,1.312129,0.771797,0.826205,0.902003,0.494335,0.786802,0.113748,0.843657,...,1.243803,4.959159,0.898815,8.049836,0.899327,2025-03-24 14:47:19.957000,P0,1,False,-16668
1,1.368466,1.414490,1.367782,0.771797,1.005129,0.902003,0.560295,0.786802,0.058877,0.843657,...,1.178277,4.003876,0.945980,6.081807,0.977698,2025-03-24 14:47:20.462000,P0,1,False,-16163
2,1.597498,1.414490,1.435598,0.771797,0.989920,0.902003,0.604796,0.786802,0.009563,0.843657,...,1.428076,4.132484,0.676362,5.573786,1.176875,2025-03-24 14:47:20.956000,P0,1,False,-15669
3,1.745149,1.414490,1.486255,0.771797,0.972852,0.902003,0.591050,0.786802,-0.142719,0.843657,...,1.276395,4.450017,0.694479,5.308864,1.629101,2025-03-24 14:47:21.457000,P0,1,False,-15168
4,1.746012,1.414490,1.238417,0.771797,0.885606,0.902003,0.509056,0.786802,0.166789,0.843657,...,1.132252,5.135644,0.743673,5.590303,1.171631,2025-03-24 14:47:21.957000,P0,1,False,-14668
5,1.351235,1.414490,0.683429,0.771797,0.447081,0.902003,0.818388,0.786802,0.332260,0.843657,...,0.750612,9.810268,5.292707,34.090560,5.772886,2025-03-24 14:47:26.457000,P0,1,False,-10168
6,1.351235,1.414490,0.683429,0.771797,0.447081,0.902003,0.818388,0.786802,0.332260,0.843657,...,0.750612,8.086873,5.292707,27.864555,5.772886,2025-03-24 14:47:26.957000,P0,1,False,-9668
7,1.351235,1.414490,0.683429,0.771797,0.447081,0.902003,0.818388,0.786802,0.332260,0.843657,...,0.750612,5.861655,5.292707,16.245310,5.772886,2025-03-24 14:47:27.456000,P0,1,False,-9169
8,1.351235,1.414490,0.683429,0.771797,0.447081,0.902003,0.818388,0.786802,0.332260,0.843657,...,0.750612,3.547116,5.292707,11.608564,5.772886,2025-03-24 14:47:27.957000,P0,1,False,-8668
9,1.351235,1.414490,0.683429,0.771797,0.447081,0.902003,0.818388,0.786802,0.332260,0.843657,...,1.893767,1.541044,1.556294,3.608325,3.700457,2025-03-24 14:47:28.470000,P0,1,False,-8155


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.366736,1.542858,1.074675,1.424260,0.861299,1.379777,0.582147,0.750607,0.260837,0.264770,...,6.549447,0.565447,1.297674,-0.089315,0.570587,2025-03-24 15:35:21.753000,P1,1,False,-13802
1,1.366736,1.514174,1.074675,1.439539,0.861299,1.319792,0.582147,0.712024,0.260837,0.391919,...,3.468387,0.542492,1.162890,-0.006784,0.284377,2025-03-24 15:35:22.253000,P1,1,False,-13302
2,1.366736,1.338750,1.074675,1.380759,0.861299,1.140598,0.582147,0.649213,0.260837,0.451724,...,4.348144,0.549239,1.269357,-0.037098,0.657135,2025-03-24 15:35:22.752000,P1,1,False,-12803
3,1.366736,1.023143,1.074675,1.176435,0.861299,1.132102,0.582147,0.570533,0.260837,0.484693,...,3.745694,0.507466,1.222264,-0.229657,0.649761,2025-03-24 15:35:23.261000,P1,1,False,-12294
4,1.366736,0.722191,1.074675,0.678832,0.861299,1.095476,0.582147,0.428500,0.260837,0.336490,...,3.112027,0.622344,1.078753,-0.196904,0.611409,2025-03-24 15:35:23.752000,P1,1,False,-11803
5,1.366736,0.728580,1.074675,0.709242,0.861299,1.163606,0.582147,0.465676,0.260837,0.313888,...,3.075364,0.692818,1.093602,-0.237440,0.337324,2025-03-24 15:35:24.253000,P1,1,False,-11302
6,1.366736,0.843297,1.074675,0.789013,0.861299,1.150964,0.582147,0.392081,0.260837,0.252126,...,2.799271,0.827084,1.035287,-0.078006,0.377024,2025-03-24 15:35:24.753000,P1,1,False,-10802
7,1.366736,0.978895,1.074675,0.919033,0.861299,1.063660,0.582147,0.433951,0.260837,0.253171,...,2.738289,0.977159,1.307699,-0.372859,0.793759,2025-03-24 15:35:25.254000,P1,1,False,-10301
8,1.366736,1.090357,1.074675,0.883014,0.861299,0.702510,0.582147,0.468556,0.260837,0.414456,...,3.285715,1.215982,1.666228,-0.318559,1.393635,2025-03-24 15:35:25.753000,P1,1,False,-9802
9,1.366736,1.158796,1.074675,0.791799,0.861299,0.170295,0.582147,0.580103,0.260837,0.485000,...,3.236932,1.332964,2.182190,-0.268376,1.700875,2025-03-24 15:35:26.258000,P1,1,False,-9297


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.252166,0.557906,1.498748,0.488383,1.183685,0.779710,1.470736,1.481641,1.223780,1.488176,...,0.894026,12.344848,17.083649,13.856881,33.266331,2025-03-25 16:05:25.068000,P2,1,True,-21868
1,0.894933,0.873525,0.791919,0.667436,1.486343,1.254148,1.033060,1.196614,1.072842,0.890497,...,0.716231,8.546360,24.433383,6.619006,15.502115,2025-03-25 16:05:38.069000,P2,1,False,-8867
2,0.890490,0.504156,0.836978,0.496453,1.479080,1.253944,1.079565,1.145566,0.870192,0.860665,...,0.780757,8.240267,20.801515,7.227439,14.537447,2025-03-25 16:05:38.569000,P2,1,False,-8367
3,0.237293,-0.054095,0.585033,0.448901,0.907757,0.595469,1.059586,0.896849,0.655197,0.631250,...,0.475301,9.976442,9.835391,4.008960,8.285409,2025-03-25 16:05:42.068000,P2,1,False,-4868
4,0.094496,-0.107220,0.473851,0.445515,1.048229,0.717805,1.083234,0.894498,0.659124,0.715490,...,0.908745,9.227968,8.357557,4.856686,10.434550,2025-03-25 16:05:42.569000,P2,1,False,-4367
5,0.144085,-0.136797,0.366727,0.338836,1.136222,0.771815,1.024174,0.894894,0.735544,0.707034,...,0.975112,9.401964,9.182514,4.778082,9.822518,2025-03-25 16:05:43.069000,P2,1,False,-3867
6,0.129325,-0.017248,0.192830,0.276657,1.107065,0.753213,0.997201,0.889442,0.791651,0.740350,...,1.147591,9.114726,9.910612,5.162643,9.689341,2025-03-25 16:05:43.587000,P2,1,False,-3349
7,0.228480,0.045845,0.065643,0.128595,1.083295,0.713088,0.909858,0.886084,0.851070,0.715969,...,1.104804,9.507164,11.500382,4.876095,8.108824,2025-03-25 16:05:44.069000,P2,1,False,-2867
8,-0.012078,-0.269448,0.152159,0.107856,1.328727,0.606678,0.927462,0.933311,0.728616,0.730567,...,0.340775,11.501908,26.381171,6.332415,21.832299,2025-03-25 16:06:39.570000,P2,4,False,-20226
9,0.702074,0.969128,0.308563,0.583530,1.039161,1.158302,1.304965,1.160354,1.197246,1.172351,...,0.434977,15.061471,13.629413,8.665964,23.200782,2025-03-25 16:06:46.070000,P2,4,False,-13726


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.472153,3.174206,3.203217,5.603670,3.187496,2025-03-25 17:53:10.464000,P3,1,False,-1
1,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.988605,0.745697,0.919937,1.332836,0.519744,2025-03-25 17:53:13.858000,P3,1,False,-34252
2,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.715902,0.657201,1.505799,1.106383,0.212720,2025-03-25 17:53:16.415000,P3,1,False,-31695
3,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.735887,1.005960,1.336621,0.924819,0.242867,2025-03-25 17:53:16.822000,P3,1,False,-31288
4,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.758752,0.999210,1.380868,0.903592,0.254801,2025-03-25 17:53:16.915000,P3,1,False,-31195
5,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.803895,0.987288,1.404743,0.908743,0.265962,2025-03-25 17:53:17.020000,P3,1,False,-31090
6,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.870930,0.984940,1.388205,0.925466,0.276484,2025-03-25 17:53:17.116000,P3,1,False,-30994
7,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.885026,0.998370,1.330233,0.948662,0.282258,2025-03-25 17:53:17.253000,P3,1,False,-30857
8,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.723364,1.078132,0.976692,1.110891,0.208673,2025-03-25 17:53:17.645000,P3,1,False,-30465
9,0.415321,0.780813,0.959144,0.525319,0.844031,0.669650,0.877933,0.568666,0.525173,0.240399,...,0.626927,1.072630,0.917765,1.126089,0.197356,2025-03-25 17:53:17.735000,P3,1,False,-30375


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,1.382039,26.085369,22.519255,20.603089,38.857508,2025-03-26 11:37:05.126000,P4,1,True,-38135
1,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.405541,30.168133,19.084177,25.350394,33.906579,2025-03-26 11:37:05.864000,P4,1,False,-37397
2,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.574645,31.821393,18.980858,23.570033,33.353280,2025-03-26 11:37:05.930000,P4,1,False,-37331
3,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.616261,33.352428,18.859268,21.927867,33.158481,2025-03-26 11:37:06.026000,P4,1,False,-37235
4,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.627268,33.498252,19.048664,21.076937,33.574622,2025-03-26 11:37:06.147000,P4,1,False,-37114
5,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.625414,31.814358,18.757982,20.600377,32.742513,2025-03-26 11:37:06.266000,P4,1,False,-36995
6,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.577348,29.725347,17.947246,20.347347,31.174622,2025-03-26 11:37:06.363000,P4,1,False,-36898
7,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,2.578640,28.224882,16.890503,20.653291,29.670605,2025-03-26 11:37:06.455000,P4,1,False,-36806
8,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,3.798758,26.652115,19.976433,17.538684,24.493523,2025-03-26 11:37:07.263000,P4,1,False,-35998
9,0.096729,0.194160,0.644864,0.211765,0.958657,0.522090,1.271278,1.259892,1.231796,1.175163,...,4.715279,24.855289,20.117128,15.922720,23.856593,2025-03-26 11:37:07.351000,P4,1,False,-35910


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,1.226269,1.444629,1.147060,1.224001,1.223358,1.248983,0.620400,0.604811,-0.061919,-0.039350,...,0.000708,0.004431,-0.000815,0.004149,0.000587,2025-03-27 13:03:24.307000,P5,1,True,-6598
1,1.238642,1.484778,1.146012,1.258805,1.235578,1.272569,0.639299,0.594300,-0.020470,-0.022165,...,0.001177,0.004733,-0.000490,0.004060,0.000643,2025-03-27 13:03:24.360000,P5,1,True,-6545
2,1.274606,1.543494,1.161744,1.322076,1.251457,1.314910,0.659406,0.577512,0.022853,-0.003746,...,0.001649,0.005011,-0.000104,0.003828,0.000895,2025-03-27 13:03:24.448000,P5,1,True,-6457
3,1.338565,1.621397,1.207561,1.403015,1.279780,1.375856,0.673247,0.557043,0.060165,0.018488,...,0.002040,0.005217,0.000276,0.003501,0.001336,2025-03-27 13:03:24.567000,P5,1,True,-6338
4,1.485384,1.842649,1.296528,1.751640,1.240559,1.315687,0.338904,0.427362,-0.005953,0.105450,...,0.007796,0.005180,0.002286,0.006563,0.001845,2025-03-27 13:03:26.081000,P5,1,False,-4824
5,1.415856,1.515901,1.214531,1.652742,1.184022,0.966684,0.334424,0.496927,-0.049188,0.194192,...,0.007891,0.005897,0.001974,0.007944,0.001096,2025-03-27 13:03:26.192000,P5,1,False,-4713
6,1.332908,1.488786,1.135063,1.671821,1.076821,0.993188,0.373497,0.518409,-0.068717,0.188370,...,0.007813,0.006592,0.001737,0.008976,0.000341,2025-03-27 13:03:26.274000,P5,1,False,-4631
7,1.241131,1.507028,1.230716,1.735831,0.999355,1.008324,0.499433,0.591854,-0.033030,0.179728,...,0.003700,0.006501,0.001414,0.008661,-0.000441,2025-03-27 13:03:27.072000,P5,1,False,-3833
8,1.199909,1.507028,1.312968,1.735831,0.999355,1.008324,0.568778,0.591854,0.005745,0.179728,...,0.007839,0.006563,0.002138,0.008979,-0.000262,2025-03-27 13:03:27.312000,P5,1,False,-3593
9,1.157094,1.507028,1.349870,1.735831,0.999355,1.008324,0.611796,0.591854,0.030961,0.179728,...,0.008083,0.006478,0.002537,0.010343,-0.000203,2025-03-27 13:03:27.393000,P5,1,False,-3512


,Delta_TP9,Delta_TP10,Theta_TP9,Theta_TP10,Alpha_TP9,Alpha_TP10,Beta_TP9,Beta_TP10,Gamma_TP9,Gamma_TP10,...,Alpha_AF8_normalized,Beta_AF7_normalized,Beta_AF8_normalized,Gamma_AF7_normalized,Gamma_AF8_normalized,local_timestamp,participant,trial_number,on_road,crossing_offset_unix_ms
0,0.985066,0.762595,0.774137,0.201074,0.997084,0.749813,0.686479,0.379442,0.349908,0.141331,...,0.672220,2.240350,2.780085,1.853243,3.294122,2025-03-27 15:21:53.751000,P6,1,True,-22027
1,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.172060,1.645343,5.297195,0.842899,3.249684,2025-03-27 15:22:06.741000,P6,1,False,-9037
2,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.357106,1.792102,4.858184,1.254488,4.760463,2025-03-27 15:22:07.265000,P6,1,False,-8513
3,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.405764,1.793515,5.070984,1.293070,4.884799,2025-03-27 15:22:07.340000,P6,1,False,-8438
4,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.419807,1.823281,5.297020,1.296757,4.950804,2025-03-27 15:22:07.426000,P6,1,False,-8352
5,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.401239,1.810208,5.220920,1.339047,4.938548,2025-03-27 15:22:07.939000,P6,1,False,-7839
6,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.397553,1.836694,5.149663,1.402509,5.086906,2025-03-27 15:22:08.025000,P6,1,False,-7753
7,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.360721,1.751278,5.757970,1.465370,5.336963,2025-03-27 15:22:08.549000,P6,1,False,-7229
8,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.305439,1.724200,5.891050,1.393393,5.119822,2025-03-27 15:22:08.661000,P6,1,False,-7117
9,1.362353,1.661185,0.969447,1.454720,1.360697,0.936773,1.024553,0.954046,0.776140,0.519771,...,0.252867,1.739670,5.896869,1.301173,4.868464,2025-03-27 15:22:08.748000,P6,1,False,-7030
